In [3]:
import xml.etree.ElementTree as et
import numpy as np
import pandas as pd
import qgrid

In [4]:
def get_root_of_xml():
    etree = et.parse('data.xml')
    return etree.getroot()

In [5]:
def get_rules_attrs(xml_root):
    rule_attr_list = [xml.attrib for xml in xml_root.iter('rule')]
    rule_attr_dict = rule_attr_list.copy()
    rule_attr_data_frame = pd.DataFrame(list(rule_attr_dict))
    return rule_attr_data_frame

In [6]:
def get_attribute_attrs(xml_root):
    attribute_attr_list = [a.attrib for a in xml_root.find('attributes').iter('name')]
    attribute_attr_dict = attribute_attr_list.copy()
    attributes_attr_data_frame = pd.DataFrame(list(attribute_attr_dict))
    attributes_attr_data_frame.drop('valueID', axis='columns', inplace=True)
    attributes_attr_data_frame.dropna(subset=['attributeID'], inplace=True)
    return attributes_attr_data_frame

In [7]:
def init_rule_matrix(rule_data_frame, attribute_data_frame):
    for x in attribute_data_frame.values:
        rule_data_frame['attribute_' + x] = np.nan
    return rule_data_frame


In [8]:
def get_rule_by_id(xml_root, id_rule):
    for rule in xml_root.iter('rule'):
        if rule.attrib['ruleID'] == id_rule:
            return rule

In [9]:
def populate_matrix_with_conditions(xml_root, data_frame):
    for i in range(len(list(data_frame.ruleID))):
        for k in list(get_rule_by_id(xml_root, data_frame.ruleID[i]).find('conditions')):
            data_frame.loc[
                data_frame['ruleID'] == data_frame.ruleID[i], str(
                    'attribute_' + list(k)[0].attrib['attributeID'])] = list(k)[2].text
    return data_frame



In [10]:
def populate_matrix_with_conclusions(xml_root, data_frame):
    for i in range(len(list(data_frame.ruleID))):
        for k in list(get_rule_by_id(xml_root, data_frame.ruleID[i]).find('conclusion')):
            data_frame.loc[
                data_frame['ruleID'] == data_frame.ruleID[i], str(
                    'attribute_' + list(k)[0].attrib['attributeID'])] = list(k)[2].text
    return data_frame


In [21]:
if __name__ == '__main__':
    root = get_root_of_xml()
    rule_attrs = get_rules_attrs(root)
    attribute_attrs = get_attribute_attrs(root)
    init_data = init_rule_matrix(rule_attrs, attribute_attrs)


    populate_matrix_with_conditions(root, init_data)
    widget = qgrid.show_grid(populate_matrix_with_conclusions(root, init_data), show_toolbar=True)
    

In [23]:
    widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [25]:
  populate_matrix_with_conclusions(root, init_data).to_excel("output.xlsx")